In [7]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, ParameterGrid
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [8]:
# training dataset loading
Training_csv_file_path = './data/TrainingSet/mRNA_sublocation_TrainingSet_NC-BERTdata.csv'
Training_data= pd.read_csv(Training_csv_file_path)
# separate sequence features and labels
X_Train = Training_data.drop(columns=['label']).values
y_Train = Training_data['label'].values
# the training data and test data are standardized
sc = StandardScaler()
sc.fit(X_Train)
X_Train = sc.transform(X_Train)

In [9]:
from sklearn.decomposition import PCA
import numpy as np
# Initialize PCA and set the number of principal components
pca = PCA(n_components=72)
# Fit PCA on the training data and transform the training data to its principal components
X_Train = pca.fit_transform(X_Train)


In [10]:
# Define hyperparameter ranges
param_grid = {
    'n_neighbors': list(range(1, 101)) 
}

# Create all combinations of hyperparameters
grid = list(ParameterGrid(param_grid))

# Initialize the result list
results = []

# 100 times 5-fold cross-validation
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=100, random_state=42)
rskf.get_n_splits(X_Train, y_Train)

# Open the CSV file for writing
with open('./result/KNN-100times-5-fold cv.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['n_neighbors', 'accuracy', 'precision', 'recall', 'f1'])
    
    # Perform a grid search
    for params in tqdm(grid, desc="Hyperparameter search"):
        acc_scores = []
        prec_scores = []
        rec_scores = []
        f1_scores = []
        
        for i, (train_index, test_index) in enumerate(rskf.split(X_Train, y_Train)):
            X_train, X_test = X_Train[train_index], X_Train[test_index]
            y_train, y_test = y_Train[train_index], y_Train[test_index]
            clf = KNeighborsClassifier(n_neighbors=params['n_neighbors'],weights='distance',algorithm='ball_tree')
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            acc_scores.append(accuracy_score(y_test, y_pred))
            prec_scores.append(precision_score(y_test, y_pred))
            rec_scores.append(recall_score(y_test, y_pred))
            f1_scores.append(f1_score(y_test, y_pred))
        
        # Save scores for this hyperparameter combination
        for acc, prec, rec, f1 in zip(acc_scores, prec_scores, rec_scores, f1_scores):
            writer.writerow([params['n_neighbors'], acc, prec, rec, f1])

        acc_mean, acc_std = np.mean(acc_scores), np.std(acc_scores)
        prec_mean, prec_std = np.mean(prec_scores), np.std(prec_scores)
        rec_mean, rec_std = np.mean(rec_scores), np.std(rec_scores)
        f1_mean, f1_std = np.mean(f1_scores), np.std(f1_scores)
        print(f" params: {params}")
        print(f" acc_mean: {acc_mean}, acc_std: {acc_std}")
        print(f" prec_mean: {prec_mean}, prec_std: {prec_std}")
        print(f" rec_mean: {rec_mean}, rec_std: {rec_std}")
        print(f" f1_mean: {f1_mean}, f1_std: {f1_std}")

Hyperparameter search:   1%|▌                                                        | 1/100 [01:25<2:20:32, 85.18s/it]

 params: {'n_neighbors': 1}
 acc_mean: 0.6243436397425812, acc_std: 0.014177773350633412
 prec_mean: 0.5729882964838081, prec_std: 0.017617391868208775
 rec_mean: 0.5363267895152952, rec_std: 0.023745239524925334
 f1_mean: 0.5538523732886395, f1_std: 0.01847594623271991


Hyperparameter search:   2%|█▏                                                       | 2/100 [02:46<2:15:37, 83.03s/it]

 params: {'n_neighbors': 2}
 acc_mean: 0.6243436397425812, acc_std: 0.014177773350633412
 prec_mean: 0.5729882964838081, prec_std: 0.017617391868208775
 rec_mean: 0.5363267895152952, rec_std: 0.023745239524925334
 f1_mean: 0.5538523732886395, f1_std: 0.01847594623271991


Hyperparameter search:   3%|█▋                                                       | 3/100 [04:08<2:13:32, 82.61s/it]

...

 params: {'n_neighbors': 98}
 acc_mean: 0.6768167317573566, acc_std: 0.012234810705573715
 prec_mean: 0.7053670589883506, prec_std: 0.02426475042734577
 rec_mean: 0.44224872590159947, rec_std: 0.02332416526673096
 f1_mean: 0.5432487511371029, f1_std: 0.020641178518950365


Hyperparameter search:  99%|██████████████████████████████████████████████████████▍| 99/100 [3:48:58<02:31, 151.39s/it]

 params: {'n_neighbors': 99}
 acc_mean: 0.6766429626154038, acc_std: 0.012313040935631339
 prec_mean: 0.7051527525200405, prec_std: 0.02425051376534058
 rec_mean: 0.4418787204281457, rec_std: 0.023834016551621436
 f1_mean: 0.5428914775900386, f1_std: 0.021023081254894608


Hyperparameter search: 100%|██████████████████████████████████████████████████████| 100/100 [3:51:28<00:00, 138.89s/it]

 params: {'n_neighbors': 100}
 acc_mean: 0.6766322031013529, acc_std: 0.012142506453026485
 prec_mean: 0.7054212746445148, prec_std: 0.024203249374940818
 rec_mean: 0.44145468588457093, rec_std: 0.023337096254860325
 f1_mean: 0.5426581802792876, f1_std: 0.020573943896870586
